### 步骤 0：通过 pip 安装 autogluon 并导入必要的包 

请确保选择 GPU 加速器

In [1]:
%%capture
!pip install autogluon.vision --pre
!pip install mxnet-cu110

In [2]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import logging
logging.basicConfig(level=logging.INFO)
# attach logging to console
logging.getLogger().addHandler(logging.StreamHandler())

# 参加比赛！
### 步骤 1：导入 autogluon

In [3]:
import autogluon.core as ag
from autogluon.vision import ImagePredictor

/opt/conda/lib/python3.7/site-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.8.0` and `torch==1.7.0` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


### 步骤 2：对比赛数据进行自定义处理

In [4]:
def fix_root(df, root='/kaggle/input/classify-leaves'):
    df['image'] = df['image'].apply(lambda x: os.path.join(root, x))
    return df

# load train data
train_data = fix_root(pd.read_csv('/kaggle/input/classify-leaves/train.csv'))

In [5]:
predictor = ImagePredictor(verbosity=1).fit(train_data, time_limit=3600*3, presets='medium_quality_faster_train', hyperparameters={'epochs': 30, 'model': 'resnest50', 'lr': ag.Categorical(0.1, 0.15)}) 
print(predictor.fit_summary())

ImagePredictor sets accuracy as default eval_metric for classification problems.
The number of requested GPUs is greater than the number of available GPUs.Reduce the number to 1


107637KB [00:03, 30316.14KB/s]                            


{'train_acc': 0.891109496124031, 'valid_acc': 0.9515250544662309, 'total_time': 5112.862143993378, 'best_config': {'model': 'resnest50', 'lr': 0.1, 'num_trials': 1, 'epochs': 30, 'batch_size': 64, 'nthreads_per_trial': 0, 'ngpus_per_trial': 8, 'time_limits': 10800, 'search_strategy': 'random', 'dist_ip_addrs': None, 'log_dir': '/kaggle/working/7801d8d5', 'searcher': 'random', 'scheduler': 'local', 'early_stop_patience': 5, 'early_stop_baseline': -inf, 'early_stop_max_value': inf, 'num_workers': 0, 'gpus': [0], 'seed': 752, 'final_fit': False, 'estimator': <class 'gluoncv.auto.estimators.image_classification.image_classification.ImageClassificationEstimator'>, 'wall_clock_tick': 1624918177.7028286, 'problem_type': 'multiclass'}, 'fit_history': {'train_acc': 0.891109496124031, 'valid_acc': 0.9515250544662309, 'total_time': 5112.862143993378, 'best_config': {'model': 'resnest50', 'lr': 0.1, 'num_trials': 1, 'epochs': 30, 'batch_size': 64, 'nthreads_per_trial': 0, 'ngpus_per_trial': 8, 'ti

### 步骤 3：测试

In [6]:
test = pd.read_csv('/kaggle/input/classify-leaves/test.csv')
test

,image
0,images/18353.jpg
1,images/18354.jpg
2,images/18355.jpg
3,images/18356.jpg
4,images/18357.jpg
...,...
8795,images/27148.jpg
8796,images/27149.jpg
8797,images/27150.jpg
8798,images/27151.jpg


In [7]:
pred_test = predictor.predict(fix_root(test.copy()))
# save prediction with image id
y = pd.concat([test, pred_test.to_frame(name='label')], axis=1)
y.to_csv('submission_test.csv', index=False)
y

,image,label
0,images/18353.jpg,asimina_triloba
1,images/18354.jpg,betula_nigra
2,images/18355.jpg,platanus_acerifolia
3,images/18356.jpg,pinus_bungeana
4,images/18357.jpg,platanus_acerifolia
...,...,...
8795,images/27148.jpg,pinus_thunbergii
8796,images/27149.jpg,sassafras_albidum
8797,images/27150.jpg,pinus_thunbergii
8798,images/27151.jpg,juniperus_virginiana


### 最后一步：保存模型和提交文件

In [8]:
predictor.save('predictor.ag')